In [2]:
import os
os.getcwd() # '/home/yanxin'
# os.listdir()
import torch
import numpy as np
from collections import defaultdict
import time
import os


import numpy as np

from tqdm import tqdm

from medicaltorch import datasets as mt_datasets
from medicaltorch import models as mt_models
from medicaltorch import transforms as mt_transforms
from medicaltorch import losses as mt_losses
from medicaltorch import metrics as mt_metrics
from medicaltorch import filters as mt_filters

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torchvision.utils as vutils

cudnn.benchmark = True
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
os.getcwd()

'/home/yanxin'

# 载入模型

In [4]:
# model = torch.load('U_model_550.pkl')
# model = torch.load('U_model_size16_100.pkl')
# model = torch.load('./sorted_data_16/U_model_size16_1e04.pkl')
model = torch.load('U_model_size16_300_1e04.pkl')

In [5]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)

In [6]:
# train_input_file=[]
# train_gt_file=[]
# test_input_file=[]
# test_gt_file=[]
# for i in range(120):
#     train_input_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm.nii')
#     train_gt_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm_roi.nii')
# for i in range(120,150):
#     test_input_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm.nii')
#     test_gt_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm_roi.nii')

In [28]:
train_input_file=[]
train_gt_file=[]
test_input_file=[]
test_gt_file=[]
# for i in range(120):
#     new_ct_name = str(i) + 'input256' + '.nii'
#     new_seg_name = str(i) + 'gt256' + '.nii'

#     train_input_file.append('/home/yanxin/testdata/' + new_ct_name)
#     train_gt_file.append('/home/yanxin/testdata/' + new_seg_name)
for i in range(120,150):
    new_ct_name = str(i) + 'input256' + '.nii'
    new_seg_name = str(i) + 'gt256' + '.nii'
    test_input_file.append('/home/yanxin/testdata/' + new_ct_name)
    test_gt_file.append('/home/yanxin/testdata/' + new_seg_name)

In [29]:
# input_file=[]
# gt_file=[]
# for i in range(150):
#     input_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm.nii')
#     gt_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm_roi.nii')
# for i in range(120,150):
#     test_input_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm.nii')
#     test_gt_file.append('/home/yanxin/data_ct/'+str(i)+'Venous_tra_5mm_roi.nii')

In [30]:
ct_num=[]
import SimpleITK as sitk
for i in test_gt_file:
    ct = sitk.ReadImage(i, sitk.sitkInt16)
    ct_array = sitk.GetArrayFromImage(ct)
    ct_num.append(ct_array.shape[0])

In [31]:
endlist=[]
a=-1
for i in range(len(ct_num)):
#     print(ct_num[i])
    a=ct_num[i]+a
    endlist.append(a)
print(endlist)

[37, 88, 133, 174, 215, 316, 362, 400, 440, 488, 528, 570, 625, 669, 709, 746, 796, 847, 933, 966, 1007, 1043, 1081, 1119, 1155, 1191, 1235, 1281, 1324, 1376]


In [32]:
startlist=[0]+[i+1 for i in endlist][:-1]

In [33]:
# train_transform = transforms.Compose([
#         mt_transforms.Resample(1.6, 1.6),
#         mt_transforms.CenterCrop2D((256, 256)),
#         mt_transforms.ToTensor(),
#         mt_transforms.NormalizeInstance(),
# ])

# val_transform = transforms.Compose([
#         mt_transforms.Resample(1.6, 1.6),
#         mt_transforms.CenterCrop2D((256, 256)),
#         mt_transforms.ToTensor(),
#         mt_transforms.NormalizeInstance(),
# ])

In [34]:
# import SimpleITK as sitk
# for ct_file in os.listdir(ct_path):

#     ct = sitk.ReadImage(os.path.join(ct_path, ct_file), sitk.sitkInt16)
#     ct_array = sitk.GetArrayFromImage(ct)

#     seg_file = ct_file.replace('volume', 'segmentation')
#     seg = sitk.ReadImage(os.path.join(seg_path, seg_file), sitk.sitkInt16)
#     seg_array = sitk.GetArrayFromImage(seg)

In [35]:
train_transform = transforms.Compose([
#         mt_transforms.Resample(1.6, 1.6),
#         mt_transforms.CenterCrop2D((256, 256)),
        mt_transforms.ToTensor(),
        mt_transforms.NormalizeInstance(),
])

val_transform = transforms.Compose([
#         mt_transforms.Resample(1.6, 1.6),
#         mt_transforms.CenterCrop2D((256, 256)),
        mt_transforms.ToTensor(),
        mt_transforms.NormalizeInstance(),
])
train_pair_tuple=[]
test_pair_tuple=[]
# for i in range(120):
#     train_pair_tuple.append((input_file[i],gt_file[i]))
for i in range(30):
    test_pair_tuple.append((test_input_file[i],test_gt_file[i]))

# load data
# 1dfd83b990ae2d3a4604b239681c9f47
# train_dataset = mt_datasets.MRI2DSegmentationDataset(train_pair_tuple,transform=train_transform)
test_dataset = mt_datasets.MRI2DSegmentationDataset(test_pair_tuple,transform=val_transform)

In [38]:
# train_loader = DataLoader(train_dataset, batch_size=32,
#                           shuffle=False, pin_memory=True,
#                           collate_fn=mt_datasets.mt_collate)

val_loader = DataLoader(test_dataset, batch_size=1,
                        shuffle=False, pin_memory=True,
                        collate_fn=mt_datasets.mt_collate)

In [39]:
import gc
gc.collect()

93

In [41]:

def Diceloss(pred, target):

    pred = pred.squeeze(dim=1)
#     print(pred.shape)

    # dice系数的定义
    dice = 2 * (pred * target).sum(dim=1).sum(dim=1).sum(dim=1) / (pred.pow(2).sum(dim=1).sum(dim=1).sum(dim=1) +
                                        target.pow(2).sum(dim=1).sum(dim=1).sum(dim=1) + 1e-5)

    # 返回的是dice距离
    return (1 - dice).mean()

In [47]:
def get_loss_perchase(start,end):
    global gt3D
    global preds3D
    input_samples, gt_samples = batch["input"], batch["gt"]# 现在一个batch是1个
    var_input = input_samples
    var_gt = gt_samples
    if i==start:
        gt3D=torch.cat((gt3D_0,var_gt),dim=1) 
    else:
        gt3D=torch.cat((gt3D,var_gt),dim=1)
    del var_gt
#     print(var_input.shape)
    preds = model(var_input)
    preds=preds.cpu().detach()
    del var_input
    if i==start:
        preds3D=torch.cat((preds3D_0,preds),dim=1) 
    else:
        preds3D=torch.cat((preds3D,preds),dim=1)
    del preds
    if i==end:
#         print(gt3D.shape)
        preds3D = threshold_predictions(preds3D)
#         preds3D = reverse0_1(preds3D)
#         gt3D = reverse0_1(gt3D)
#         loss=Diceloss(gt3D, preds3D)
#         print('Diceloss',loss.item())
#         print(gt3D.sum(dim=1).sum(dim=1).sum(dim=1))
        
        loss = mt_losses.dice_loss(gt3D, preds3D)
        print('mt_losses.dice_loss',loss.item())
        losslist.append(loss.item())
def threshold_predictions(predictions, thr=0.999):
    thresholded_preds = predictions[:]
    low_values_indices = thresholded_preds < thr
    thresholded_preds[low_values_indices] = 0
    low_values_indices = thresholded_preds >= thr
    thresholded_preds[low_values_indices] = 1
    return thresholded_preds
def reverse0_1(predictions,):
    predictions[:]=1-predictions[:]
#     thresholded_preds = predictions[:]
#     thresholded_preds_copy = thresholded_preds.copy()
#     low_values_indices = thresholded_preds == 1
#     thresholded_preds_copy[low_values_indices] = 0
#     low_values_indices = thresholded_preds ==0
#     thresholded_preds_copy[low_values_indices] = 1
    return predictions

# 验证集有30个，观察每个的diceloss

In [48]:
losslist=[]
gt3D_0 =torch.zeros(1,1,256,256,)
preds3D_0 =torch.zeros(1,1,256,256, )
global get3D
for i, batch in enumerate(val_loader):
    if i<(endlist[0]+1): get_loss_perchase(startlist[0],endlist[0])
    elif i<(endlist[1]+1): get_loss_perchase(startlist[1],endlist[1])
    elif i<(endlist[2]+1): get_loss_perchase(startlist[2],endlist[2])
    elif i < (endlist[3] + 1): get_loss_perchase(startlist[3], endlist[3])
    elif i < (endlist[4] + 1): get_loss_perchase(startlist[4], endlist[4])
    elif i < (endlist[5] + 1): get_loss_perchase(startlist[5], endlist[5])
    elif i < (endlist[6] + 1): get_loss_perchase(startlist[6], endlist[6])
    elif i < (endlist[7] + 1): get_loss_perchase(startlist[7], endlist[7])
    elif i < (endlist[8] + 1): get_loss_perchase(startlist[8], endlist[8])
    elif i < (endlist[9] + 1): get_loss_perchase(startlist[9], endlist[9])
    elif i < (endlist[10] + 1): get_loss_perchase(startlist[10], endlist[10])
    elif i < (endlist[11] + 1): get_loss_perchase(startlist[11], endlist[11])
    elif i < (endlist[12] + 1): get_loss_perchase(startlist[12], endlist[12])
    elif i < (endlist[13] + 1): get_loss_perchase(startlist[13], endlist[13])
    elif i < (endlist[14] + 1): get_loss_perchase(startlist[14], endlist[14])
    elif i < (endlist[15] + 1): get_loss_perchase(startlist[15], endlist[15])
    elif i < (endlist[16] + 1): get_loss_perchase(startlist[16], endlist[16])
    elif i < (endlist[17] + 1): get_loss_perchase(startlist[17], endlist[17])
    elif i < (endlist[18] + 1): get_loss_perchase(startlist[18], endlist[18])
    elif i < (endlist[19] + 1): get_loss_perchase(startlist[19], endlist[19])
    elif i < (endlist[20] + 1): get_loss_perchase(startlist[20], endlist[20])
    elif i < (endlist[21] + 1): get_loss_perchase(startlist[21], endlist[21])
    elif i < (endlist[22] + 1): get_loss_perchase(startlist[22], endlist[22])
    elif i < (endlist[23] + 1): get_loss_perchase(startlist[23], endlist[23])
    elif i < (endlist[24] + 1): get_loss_perchase(startlist[24], endlist[24])
    elif i < (endlist[25] + 1): get_loss_perchase(startlist[25], endlist[25])
    elif i < (endlist[26] + 1): get_loss_perchase(startlist[26], endlist[26])
    elif i < (endlist[27] + 1): get_loss_perchase(startlist[27], endlist[27])
    elif i < (endlist[28] + 1): get_loss_perchase(startlist[28], endlist[28])
    elif i < (endlist[29] + 1): get_loss_perchase(startlist[29], endlist[29])



/home/yanxin/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/home/yanxin/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/yanxin/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


mt_losses.dice_loss -0.6198347210884094
mt_losses.dice_loss -1.6556288073843461e-07
mt_losses.dice_loss -0.888526201248169
mt_losses.dice_loss -0.36709997057914734
mt_losses.dice_loss -0.6019771695137024
mt_losses.dice_loss -0.6509455442428589
mt_losses.dice_loss -0.579397976398468
mt_losses.dice_loss -0.8073201179504395
mt_losses.dice_loss -0.7317410707473755
mt_losses.dice_loss -0.9153534770011902
mt_losses.dice_loss -0.006111546419560909
mt_losses.dice_loss -0.365709125995636
mt_losses.dice_loss -0.48696786165237427
mt_losses.dice_loss -4.629627312624507e-07
mt_losses.dice_loss -0.1651732325553894
mt_losses.dice_loss -0.7957375645637512
mt_losses.dice_loss -0.8286338448524475
mt_losses.dice_loss -0.8817884922027588
mt_losses.dice_loss -0.6009641289710999
mt_losses.dice_loss -0.2629691958427429
mt_losses.dice_loss -0.7922093272209167
mt_losses.dice_loss -0.7286822199821472
mt_losses.dice_loss -0.7116897702217102
mt_losses.dice_loss -0.10497540235519409
mt_losses.dice_loss -0.65930783

# dice_loss的一些统计特征

In [6]:
print(np.mean(losslist))
print(np.max(losslist))
print(np.min(losslist))
print(np.std(losslist))
#550
# -0.5144157908650248
# -8.795073824785504e-08
# -0.9419780969619751
# 0.32080316267711895
#1300
# -0.5176453462473692
# -2.3310018093525287e-07
# -0.935321033000946
# 0.3239882057457576
#sort16 100  #这个居然表现不一致，不过想一想230轮确实ta
# 这个采样之后表现不一致的解释，就是对没有病的那些图片不能充分学习，都看成了有病的，所以采样率要提升
# -0.44041477211152935
# -1.9379839955036005e-07
# -0.8532313108444214
# 0.27906281110196524
#sort16 230
# -0.4890719051486059
# -2.4330893211299554e-07
# -0.7980189919471741
# 0.2549805772882429
#sort16 500 1E04
# -0.4737533453806435
# -2.4330893211299554e-07
# -0.8478531837463379
# 0.2665796454403776
#sort16 150 1E04
# -0.5472723477884576
# -2.141327115623426e-07
# -0.8840871453285217
# 0.28079788497351005
#sort16 200 1E04
# -0.5661413162339097
# -2.0618551843654132e-07
# -0.8950169086456299
# 0.2827369643951482
#sort16 300 1E04
# -0.5762936150616108
# -1.6556288073843461e-07
# -0.9153534770011902
# 0.2830451183270229
#sort16 1000 1E04
# -0.5441274456479097
# -2.4330893211299554e-07
# -0.908866822719574
# 0.285619097358927

0.48265483405377296
0.9251338427679626
0.0
0.317014220348816


In [50]:
import gc
gc.collect()

0

In [51]:
def get_index(i,endlist):
    for m in endlist:
        if i>m and i<=endlist[endlist.index(m)+1]:
            return endlist.index(m)

# 画图

In [1]:
for i, batch in enumerate(val_loader):
    print(get_index(i,endlist))
#     (endlist)
#     if i<40:
#         endlist
#         print(i)
        #     print(i,batch.shape)
    input_samples, gt_samples = batch["input"], batch["gt"]# 现在一个batch是四个
    var_input = input_samples.cuda()
    var_gt = gt_samples.cuda()
    preds = model(var_input)
    preds = threshold_predictions(preds)
#         preds = reverse0_1(preds)
#         var_gt = reverse0_1(var_gt)
#     preds = model(input_samples)
#         print(preds)
    aa=var_input.cpu().detach().numpy()
    a=preds.cpu().detach().numpy()
#         print(a.shape)
    b=var_gt.cpu().detach().numpy()
    if b[0][0].max()==1 or a[0][0].max()==1:
        plt.figure(figsize=(15, 15))
    #         print(a[0].shape)
    #         print(b.shape)
    #         print(a[0][0])
        plt.subplot(1,2,1)
        plt.imshow(aa[0][0],cmap='gray')
        plt.subplot(1,2,2)
        plt.imshow(marked_roi(aa[0][0],a[0][0],b[0][0]))
#             plt.subplot(2,2,3)
#             plt.imshow(b[0][0],cmap='gray')# gt
#             plt.subplot(2,2,4)
#             plt.imshow(a[0][0],cmap='gray')# preds
#             plt.subplot(2,2,4)
#             plt.imshow(marked_roi(aa[0][0],a[0][0],b[0][0]))
        plt.show()

NameError: name 'val_loader' is not defined

In [2]:
from skimage import io, measure, draw,color
import skimage
# from parallel import parallel_process, agg_results
from multiprocessing import Manager
# from config import Config
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
def marked_roi(image, mask_pred,mask_gt):
    _min = image.min()
    _max = image.max()
    image = (image - _min) / (_max - _min)
    image = skimage.color.gray2rgb(image)
    contour_gt = measure.find_contours(mask_gt, 0)
    contour_pred = measure.find_contours(mask_pred, 0)
#     print(len(contour_gt))
#     print(len(contour_pred))
    if len(contour_gt)==0 and len(contour_pred)==0:
        return image
    if len(contour_gt) >= 1:
        for gt_lk in contour_gt:
            contour_pixel_gt = (np.array([int(c[0]) for c in gt_lk]), np.array([int(c[1]) for c in gt_lk]))
#             marked = image.copy()
            image[contour_pixel_gt] = [1, 0, 0] # 目标是红
    if len(contour_pred) >= 1:
        for pred_lk in contour_pred:
            contour_pixel_pred = (np.array([int(c[0]) for c in pred_lk]), np.array([int(c[1]) for c in pred_lk]))
#             marked = marked.copy()
            image[contour_pixel_pred] = [0, 0, 1]
#         contour_pixel_pred = (np.array([int(c[0]) for c in contour_pred[0]]), np.array([int(c[1]) for c in contour_pred[0]]))
#     print(len(contour_pixel))
#         marked[contour_pixel_pred] = [1, 0, 0]
    return image

In [3]:
plt.figure(figsize=(15,15))
plt.imshow(marked_roi(aa[0][0],a[0][0],b[0][0]))

NameError: name 'aa' is not defined

<Figure size 1080x1080 with 0 Axes>